# Step 0.1. Import necessary libraries 

In [ ]:
# Standard python libraries
import logging
import os
import time

# Installed libraries
import numpy as np
import pandas as pd
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
import torch
from matplotlib import pyplot as plt

# Imports from our package
from lightautoml.automl.base import AutoML
from lightautoml.ml_algo.boost_lgbm import BoostLGBM
from lightautoml.ml_algo.tuning.optuna import OptunaTuner
from lightautoml.pipelines.features.lgb_pipeline import LGBSimpleFeatures, LGBSeqSimpleFeatures, LGBMultiSeqSimpleFeatures
from lightautoml.pipelines.ml.base import MLPipeline
from lightautoml.pipelines.selection.importance_based import ImportanceCutoffSelector, ModelBasedImportanceEstimator
from lightautoml.reader.base import PandasToPandasReader, DictToNumpySeqReader
from lightautoml.tasks import Task
from lightautoml.automl.blend import WeightedBlender
from lightautoml.dataset.seq_np_pd_dataset import SeqNumpyPandasDataset
from lightautoml.dataset.roles import NumericRole, CategoryRole, DatetimeRole
from lightautoml.dataset.np_pd_dataset import NumpyDataset, PandasDataset
from lightautoml.ml_algo.random_forest import RandomForestSklearn
from lightautoml.validation.np_iterators import TimeSeriesIterator

In [ ]:
DATA_FOLDER = '/home/simakovde/hdd/kaggle/idao21_final/idao_2021_train'
train = pd.read_csv(f'{DATA_FOLDER}/funnel.csv' )

clients = pd.read_csv(f'{DATA_FOLDER}/client.csv')
deals = pd.read_csv(f'{DATA_FOLDER}/deals.csv')
aum = pd.read_csv(f'{DATA_FOLDER}/aum.csv')
trxn = pd.read_csv(f'{DATA_FOLDER}/trxn.csv')
com = pd.read_csv(f'{DATA_FOLDER}/com.csv')
appl = pd.read_csv(f'{DATA_FOLDER}/appl.csv')
payments = pd.read_csv(f'{DATA_FOLDER}/payments.csv')
balance = pd.read_csv(f'{DATA_FOLDER}/balance.csv')

In [ ]:
train.shape, clients.shape, deals.shape, aum.shape, trxn.shape, com.shape, appl.shape, payments.shape, balance.shape

In [ ]:
train.head()

In [ ]:
clients.head()

In [ ]:
balance.head()

In [ ]:
# в доп таблицах только данные текущих актуальных клиентов <- реализован
# в доп таблицах данные для всех клиентов
# в доп таблицах данные только для новых клиентов 

In [ ]:
seq_params = {'clients': {'case': 'ids',
                          'params': {},
                          'scheme': {'to': 'plain', 'from_id': 'client_id', 'to_id': 'client_id'}
                         }
             }

X_train = {'plain':train , 'seq': None}
task = Task('binary', metric='logloss')

roles={'target': 'sale_flg', 'drop': ['sale_amount', 'contacts']}

reader = DictToNumpySeqReader(task=task, cv=3, seq_params=seq_params)

feats = LGBMultiSeqSimpleFeatures()
model = BoostLGBM()
pipeline_lvl1 = MLPipeline([model], pre_selection=None, features_pipeline=feats, post_selection=None)

automl = AutoML(reader, [
    [pipeline_lvl1],
], skip_conn=False)

oof_pred = automl.fit_predict(X_train, roles=roles, verbose=4)

metric_l = log_loss(train[roles['target']], oof_pred.data[:, 0])
metric_a = roc_auc_score(train[roles['target']], oof_pred.data[:, 0])

print('=============')
print(f'log-loss: {metric_l}, roc_auc: {metric_a}')

_pred = automl.predict(X_train)


In [ ]:
%%time
seq_params = {'trxn': {'case': 'ids',
                       'params': {},
                       'scheme': {'to': 'plain', 'from_id': 'client_id', 'to_id': 'client_id'}
                       }
             }

X_train = {'plain':train , 'seq': {'trxn': trxn}}

task = Task('binary', metric='logloss')

roles={'target': 'sale_flg', 'drop': ['sale_amount', 'contacts']}

reader = DictToNumpySeqReader(task=task, cv=3, seq_params=seq_params)

feats = LGBMultiSeqSimpleFeatures()
model = BoostLGBM()
pipeline_lvl1 = MLPipeline([model], pre_selection=None, features_pipeline=feats, post_selection=None)

automl = AutoML(reader, [
    [pipeline_lvl1],
], skip_conn=False)

oof_pred = automl.fit_predict(X_train, roles=roles, verbose=3)

metric_l = log_loss(train[roles['target']], oof_pred.data[:, 0])
metric_a = roc_auc_score(train[roles['target']], oof_pred.data[:, 0])

print('=============')
print(f'log-loss: {metric_l}, roc_auc: {metric_a}')

_pred = automl.predict(X_train)


In [ ]:
%%time
seq_params = {'clients': {'case': 'ids',
                       'params': {},
                       'scheme': {'to': 'plain', 'from_id': 'client_id', 'to_id': 'client_id', 'type': 'lookup'}
                         }
             }

X_train = {'plain':train , 'seq': {'clients': clients}}
task = Task('binary', metric='logloss')

roles={'target': 'sale_flg', 'drop': ['sale_amount', 'contacts']}

reader = DictToNumpySeqReader(task=task, cv=3, seq_params=seq_params)

feats = LGBMultiSeqSimpleFeatures()
model = BoostLGBM()
pipeline_lvl1 = MLPipeline([model], pre_selection=None, features_pipeline=feats, post_selection=None)

automl = AutoML(reader, [
    [pipeline_lvl1],
], skip_conn=False)

oof_pred = automl.fit_predict(X_train, roles=roles, verbose=3)

metric_l = log_loss(train[roles['target']], oof_pred.data[:, 0])
metric_a = roc_auc_score(train[roles['target']], oof_pred.data[:, 0])

print('=============')
print(f'log-loss: {metric_l}, roc_auc: {metric_a}')

_pred = automl.predict(X_train)


In [ ]:
%%time
seq_params = {'balance': {'case': 'ids',
                       'params': {},
                       'scheme': {'to': 'plain', 'from_id': 'client_id', 'to_id': 'client_id'}
                       }, 
              'payments': {'case': 'ids',
                       'params': {},
                       'scheme': {'to': 'plain', 'from_id': 'client_id', 'to_id': 'client_id'}
                       }, 
              'appl': {'case': 'ids',
                       'params': {},
                       'scheme': {'to': 'plain', 'from_id': 'client_id', 'to_id': 'client_id'}
                       },
              'com': {'case': 'ids',
                       'params': {},
                       'scheme': {'to': 'plain', 'from_id': 'client_id', 'to_id': 'client_id'}
                       },
              'trxn': {'case': 'ids',
                       'params': {},
                       'scheme': {'to': 'plain', 'from_id': 'client_id', 'to_id': 'client_id'}
                       },
              'aum': {'case': 'ids',
                       'params': {},
                       'scheme': {'to': 'plain', 'from_id': 'client_id', 'to_id': 'client_id'}
                       },
              'deals': {'case': 'ids',
                       'params': {},
                       'scheme': {'to': 'plain', 'from_id': 'client_id', 'to_id': 'client_id'}
                       },
              'clients': {'case': 'ids',
                       'params': {},
                       'scheme': {'to': 'plain', 'from_id': 'client_id', 'to_id': 'client_id', 'type': 'lookup'}
                       }
             }
X_train = {'plain':train ,
           'seq': {'balance': balance,
                   'payments': payments,
                   'appl': appl,
                   'com': com,
                   'trxn': trxn,
                   'aum': aum,
                   'deals': deals,
                   'clients': clients}}

task = Task('binary', metric='logloss')

roles={'target': 'sale_flg', 'drop': ['sale_amount', 'contacts']}

reader = DictToNumpySeqReader(task=task, cv=3, seq_params=seq_params)

feats = LGBMultiSeqSimpleFeatures()
model = BoostLGBM()
pipeline_lvl1 = MLPipeline([model], pre_selection=None, features_pipeline=feats, post_selection=None)

automl = AutoML(reader, [
    [pipeline_lvl1],
], skip_conn=False)

oof_pred = automl.fit_predict(X_train, roles=roles, verbose=3)

metric_l = log_loss(train[roles['target']], oof_pred.data[:, 0])
metric_a = roc_auc_score(train[roles['target']], oof_pred.data[:, 0])

print('=============')
print(f'log-loss: {metric_l}, roc_auc: {metric_a}')

_pred = automl.predict(X_train)